In [148]:
import pandas as pd
import sqlite3


In [149]:
#Credit to CS207

def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [col[i] for col in q]
    return pd.DataFrame.from_dict(framelist)

In [150]:
db = sqlite3.connect('data/database.sqlite')
cursor = db.cursor()

In [151]:
match_cols = [col[1] for col in cursor.execute("PRAGMA table_info(Match)")]
player_cols = [col[1] for col in cursor.execute("PRAGMA table_info(Player)")]
player_attr_cols = [col[1] for col in cursor.execute("PRAGMA table_info(Player_Attributes)")]

In [152]:
query = '''SELECT * FROM Match'''
match = viz_tables(match_cols, query)

In [154]:
query = '''SELECT * FROM Player'''
player = viz_tables(player_cols, query)

In [155]:
query = '''SELECT * FROM Player_Attributes'''
player_attr = viz_tables(player_attr_cols, query)

In [156]:
columns = list(player_attr.columns.drop(['id', 'player_fifa_api_id', 'player_api_id', 'date']))

In [157]:
def get_latest_rating_date(player_api_id, columns = columns):
    #Get row of latest rating. Ignore date in name
    #Merge might take longer, so just do it this way
    if pd.isnull(player_api_id):
        df = pd.DataFrame(columns = columns)
        df.loc[0] = [None for column in columns]
        return df
    player = player_attr[player_attr['player_api_id'] == player_api_id]
    return player.loc[player.date == player.date.max(), columns]

In [158]:
get_latest_rating_date(505942)

overall_rating  potential preferred_foot attacking_work_rate  \
0            67.0       71.0          right              medium   

  defensive_work_rate  crossing  finishing  heading_accuracy  short_passing  \
0              medium      49.0       44.0              71.0           61.0   

   volleys     ...       vision  penalties  marking  standing_tackle  \
0     44.0     ...         54.0       48.0     65.0             69.0   

   sliding_tackle  gk_diving  gk_handling  gk_kicking  gk_positioning  \
0            69.0        6.0         11.0        10.0             8.0   

   gk_reflexes  
0          8.0  

[1 rows x 38 columns]

In [160]:
for i in range(1, 12):
    home_column_names = ['home_player_{}_'.format(i) + column for column in columns]
    away_column_names = ['away_player_{}_'.format(i) + column for column in columns]
    #https://stackoverflow.com/questions/30471101/pandas-convert-series-of-dataframes-to-single-dataframe
    home_player_columns = match['home_player_{}'.format(i)].apply(get_latest_rating_date)
    home_player_columns = pd.concat([r for r in home_player_columns], ignore_index=True)
    home_player_columns.columns = home_column_names
    away_player_columns = match['away_player_{}'.format(i)].apply(get_latest_rating_date)
    away_player_columns = pd.concat([r for r in away_player_columns], ignore_index=True)  
    away_player_columns.columns = away_column_names
    match = match.merge(home_player_columns, left_index= True, right_index = True)
    match = match.merge(away_player_columns, left_index= True, right_index = True)

In [161]:
#x is home, y is away
match.to_csv("data/cleaned/european_matches.csv", index = False)

In [162]:
match.shape

(25979, 989)

In [163]:
db.close()